# Landsattrend data preparation script for clowder extractor

In [1]:
import ee#, eemont
#ee.Authenticate()
ee.Initialize()

import geemap
from importlib import reload  
import geopandas as gpd

from GEE_HotSpot.modules import high_level_functions
from GEE_HotSpot.modules import utils_Landsat_SR as utils_LS
from GEE_HotSpot.modules import ms_indices as indices
from GEE_HotSpot.modules import configs, utils_string

import numpy as np

In [11]:
def get_utmzone_from_lon(lon):
    return int(31 + np.floor(lon/ 6))

def crs_from_utmzone(utm):
    return f'EPSG:326{utm:02d}'

def epsg_from_utmzone(utm):
    return f'326{utm:02d}'


# change naming to include
# 1.Zone (e.g.04N)
# 2. Latitude (- or direction N/S)
# 3. Longitude (- or direction W/E)
def prefix_from_utmzone(utm):
    return f'trendimage_Z{utm:02d}'

def create_dem_data():
    # Create DEM data from various sources
    alosdem = ee.ImageCollection("JAXA/ALOS/AW3D30/V3_2").mosaic().select(['DSM'], ['elevation'])
    alosdem = alosdem.addBands(ee.Terrain.slope(alosdem)).select(['elevation', 'slope']).toFloat()
    
    nasadem = ee.Image("NASA/NASADEM_HGT/001").select(['elevation'])
    nasadem = nasadem.addBands(ee.Terrain.slope(nasadem)).select(['elevation', 'slope']).toFloat()
    
    arcticDEM = ee.Image("UMN/PGC/ArcticDEM/V3/2m_mosaic").select(['elevation'])
    arcticDEM = arcticDEM.addBands(ee.Terrain.slope(arcticDEM)).select(['elevation', 'slope']).toFloat()
    
    dem = ee.ImageCollection([arcticDEM, alosdem, nasadem]).mosaic()
    return dem

def run_preprocess(config_trend, crs='EPSG:32656', prefix='trendimage_Z056-Kolyma', tasking=True):

    config_trend['geom'] = geom
    trend = high_level_functions.runTCTrend(config_trend)
    data = trend['data']

    #### setup data
    dem = create_dem_data()
    data_export = data.addBands(dem).toFloat().select(data_cols)
    if tasking:
        ### Export
        task = ee.batch.Export.image.toDrive(
            image=data_export,
            description=prefix,
            folder=OUTFOLDER,
            fileNamePrefix=prefix,
            crs=crs,
            region=geom,
            scale=30,
            maxPixels=1e12)
        task.start()
        return None
    else:
        return data_export

def get_lon_from_utmzone(zone, distance):
    start = (zone-31)*6
    return list(range(start, start+6, distance))

def epsgprefix_from_utmzone(utm):
    epsg = epsg_from_utmzone(utm)
    return f'trendimage_{epsg}'

def make_fileprefix(epsg, period_start, period_end, lon, lat):
    fileprefix = f'trendimage_{period_start}-{period_end}_{epsg}_{lon}_{lat}'
    return fileprefix

In [12]:
data_cols = ['TCB_slope',
             'TCB_offset',
             'TCG_slope', 
             'TCG_offset', 
             'TCW_slope',
             'TCW_offset',
             'NDVI_slope',
             'NDVI_offset',
             'NDMI_slope',
             'NDMI_offset',
             'NDWI_slope',
             'NDWI_offset', 
             'elevation', 
             'slope'] 

### Run  Batch Process

#### Project Properties - Change Settings Here 

In [19]:
# PROPERTIES
# SET METADATA AND EXPORT PARAMETERS
MAXCLOUD = 70
STARTYEAR = 2000
ENDYEAR = 2020
STARTMONTH = 7
ENDMONTH = 8
SCALE = 30
OUTFOLDER = 'PDG_Trend'

# SET SPATIAL PARAMETERS
UTM_ZONE = 17
LATITUDE_MIN = 70
LATITUDE_MAX = 72

# SPATIAL TILING SETTINGS (can be kept as is!)
X_OVERLAP = 0.1
X_SIZE_BUFFERED = 3 + (2*X_OVERLAP)
Y_SIZE = 1
X_SIZE = 3

In [14]:
# image metadata Filters
config_trend = {
  'date_filter_yr' : ee.Filter.calendarRange(STARTYEAR, ENDYEAR, 'year'),
  'date_filter_mth' : ee.Filter.calendarRange(STARTMONTH, ENDMONTH, 'month'),
  'meta_filter_cld' : ee.Filter.lt('CLOUD_COVER', MAXCLOUD),
  'select_bands_visible' : ["B1", "B2","B3","B4"],
  'select_indices' : ["TCB", "TCG", "TCW", "NDVI", "NDMI", "NDWI"],
  'select_TCtrend_bands' : ["TCB_slope", "TCG_slope", "TCW_slope"],
  'geom' : None
}
#------ RUN FULL PROCESS FOR ALL REGIONS IN LOOP ------------------------------
#Map.addLayer(imageCollection, {}, 'TCVIS')

In [16]:
#longitudes = range(90, 96, X_SIZE)
longitudes = get_lon_from_utmzone(UTM_ZONE, X_SIZE)
print(list(longitudes))

latitudes = range(LATITUDE_MIN, LATITUDE_MAX, Y_SIZE)
print(list(latitudes))

[-84, -81]
[70, 71]


#### Run Processing 

In [18]:
for X_MIN in longitudes:
    for Y_MIN in latitudes:
        geom = ee.Geometry.Rectangle(coords=[X_MIN-X_OVERLAP, Y_MIN, X_MIN+X_SIZE_BUFFERED, Y_MIN+Y_SIZE], proj=ee.Projection('EPSG:4326'))
        config_trend['geom'] = geom
        utm = get_utmzone_from_lon(X_MIN)
        
        crs = crs_from_utmzone(utm)
        epsg = epsg_from_utmzone(utm)
        prefix = make_fileprefix(epsg=epsg, period_start=STARTYEAR, period_end=ENDYEAR, lon=X_MIN, lat=Y_MIN)
        print(prefix)
        
        run_preprocess(config_trend, crs=crs, prefix=prefix)

trendimage_2000-2020_32617_-84_70
['B1', 'B2', 'B3', 'B4']
['TCB', 'TCG', 'TCW', 'NDVI', 'NDMI', 'NDWI']
trendimage_2000-2020_32617_-84_71
['B1', 'B2', 'B3', 'B4']
['TCB', 'TCG', 'TCW', 'NDVI', 'NDMI', 'NDWI']
trendimage_2000-2020_32617_-81_70
['B1', 'B2', 'B3', 'B4']
['TCB', 'TCG', 'TCW', 'NDVI', 'NDMI', 'NDWI']
trendimage_2000-2020_32617_-81_71
['B1', 'B2', 'B3', 'B4']
['TCB', 'TCG', 'TCW', 'NDVI', 'NDMI', 'NDWI']


In [ ]:
Map = geemap.Map()
Map.add_ee_layer(geom)
Map.center_object(geom)

In [ ]:
Map